In [1]:
#import library yang digunakan
import csv
import pandas as pd
from itertools import combinations
from time import time

#Membaca data yang akan digunakan
df = pd.read_csv('dataset.csv', sep=";")
df.head()

#Preprocessing data membuang kolom yang tidak diperlukan
grouped_df = df.groupby(by = ['TID'])
 
menu = []
for group, pdf in grouped_df:
    menu.append(pdf['Menu'].values.tolist())

#Mengubah list menjadi dataframe
data = pd.DataFrame(menu)

#Input nilai minimum support dan confidence
minsup=float(input("Masukkan nilai minimum support: "))
minsup=minsup*len(data)
minconf=float(input("Masukkan nilai minimum confidence: "))

#Mulai menghitung execution time
t1 = time()

#Data preprocessing merubah data menjadi list
items = []  
for i in range(0, len(data)):  
    items.append([str(data.values[i,j]) for j in range(0, len(data.values[0]))])

#Membuat daftar kosakata yang terdapat pada list
count = [dict() for x in range(len(data.values[0])+1)]

#Menghitung nilai support item secara satuan
s=[]
for i in items:
    for j in i:
        s.append(j)
for i in s:
    #Jika item terdapat dalam daftar kosakata, tambahkan 1 hitungan
    if i in count[1]:
        count[1][i] = count[1][i] + 1
    #Jika item tidak terdapat dalam daftar kosakata, hitung menjadi 1
    else:
        count[1][i] = 1
        
#Menghapus data yang kosong(nan values) dan infrequent items
for i in count[1].copy():
    if(count[1][i]<minsup):
        count[1].pop(i)
count[1].pop('None')

#Menyimpan transaksi yang baru tanpa infrequent items
a=list(count[1])
item=[list() for i in range(len(data))]
c=0
for i in range(0,len(items)):
    for j in range(len(items[i])):
        if(a.__contains__(items[i][j])!=0):
            item[i].append(items[i][j])
            
#Fungsi untuk mengurutkan list berdasarkan nilai support
def sort(a):
    for i in range(len(a)-1):
        for j in range(len(a)-i-1):
            if(count[1][a[j]]<count[1][a[j+1]]):
                a[j],a[j+1]=a[j+1],a[j]

#Memanggil fungsi untuk mengurutkan list secara descending berdasarkan nilai support
for i in range(0,len(data)):
    if(len(item[i])>1):
        sort(item[i])
        
#Algoritma FP Tree
class tree:
    def __init__(self, name, sup, parent):
        self.name = name
        self.sup = sup
        self.nodeLink = None
        self.parent = parent
        self.children = []

#Fungsi untuk cek suatu node merupakan anak dari node sebelumnya
def ispresent(node,name):
    f=-1
    for i in node.children:
        f=f+1
        if(i.name==name):
            return f
    return -1

#HeaderTable yang digunakan untuk sebagai referensi transaksi pertama/terakhir, dijadikan sebagai linked list untuk membuat candidate tree
lastocc=count[1].copy()
for i in lastocc:
    lastocc[i]=None

#Fungsi untuk membuat FP-Tree
root = tree("root",-1,None)
z=0
for i in item:
    current=root
    for j in range(len(i)):
        if(ispresent(current,i[j])>=0):
            current=current.children[ispresent(current,i[j])]
            current.sup=current.sup+1
        else:
            child=tree(i[j],1,current)
            current.children.append(child)
            t=current
            current=current.children[ispresent(current,i[j])]
            current.parent=t
            if(lastocc[current.name]==None):
                lastocc[current.name]=current
            else:
                current.nodeLink=lastocc[current.name]
                lastocc[current.name]=current

#Fungsi untuk ekstraksi sebuah itemset dari tuple
def value(a):
    a=str(a)
    a=a[:-2]
    a=a[2:]
    a=a[:-1]
    return a

#Fungsi untuk mendapatkan itemset yang sering muncul, dengan akhiran 'node' dan length n
def singlepath(node,n):
    c=0
    sup=node.sup
    path=[]
    pathname=[]
    current=node
    
    #Dapatkan jalur dari node terbaru hingga ke root
    while(current.parent!=None):
        path.append(current)
        pathname.append(current.name)
        current=current.parent  
    path.remove(node)
    pathname.remove(node.name)
    candidatepath=[]
    temp_candidatepath=[]
    #Menghasilkan combinasi length n pada sebuah path
    a =(list(combinations(pathname,n)))
    for j in a:
        temp_candidatepath.append(tuple(sorted(j)))
    #Tambahkan akhiran 'node.name' pada path sebelumnya
    for j in temp_candidatepath:
        j=list(j)
        j.append(node.name)
        candidatepath.append(sorted(j))
    #Perbarui jumlah itemset yang dihasilkan
    for j in candidatepath:
        j=tuple(j)
        if j in count[n+1]:
            count[n+1][j]=count[n+1][j]+sup
        else:
            count[n+1][j]=sup
            
    #Iterasi candidate tree secara rekursif 
    if(node.nodeLink!=None):
        node=node.nodeLink
        singlepath(node,i)
    
#Cek itemset yang frequent
def frequent(n):
    f=0
    for i in count[n]:
        if(count[n][i]>=minsup):
            f=1
    if(f==1):
        return 1
    else:
        return 0

#Memanggil fungsi singlepath untuk node yang frequent
for i in range(1,len(data.values[0])+1): 
    if(frequent(i)==1):
        for j in lastocc:
            singlepath(lastocc[j],i)

#Menghapus infrequent itemset
for z in range(len(data.values[0])+1):            
    for i in count[z].copy():
            if(count[z][i]<minsup):
                count[z].pop(i)

#Get 'q', length dari itemset terpanjang
i=2
while(len(count[i-1])!=0):
   i=i+1
q=i-2
#q=len(count)-1

#Mencari itemset yang maksimal dan dekat
maximal=[]
closed=[]
for i in range(1,q):
    for j in count[i]:
        fm=0
        fc=0
        for k in count[i+1]:
            a=set(list([j]))
            b=set(list(k))
            #Set menjadi maksimal jika tidak ada superset yang frequent
            if(a.intersection(b)==a):
                fm=1
                #Set tertutup jika superset tidak memiliki nilai support yang sama
                if(count[i][j]==count[i+1][k]):
                    fc=1
        if(fm==0):
            maximal.append(j)
        if(fc==0):
            closed.append(j)
#Semua set yang terdapat pada pohon bagian atas, otomatis menjadi itemset yang maksimal dan dekat
for i in count[q]:
    maximal.append(i)
    closed.append(i)

#Menemukan Association Rules
c=0
a=[]
item_a=[]
b=[]
item_b=[]
support=[]
confidence=[]
lift_ratio=[]
ant=count.copy()
for i in range(q,0,-1):
    for j in ant[i]:
        for k in range(i-1,0,-1):
            s=list(combinations(list(j),k))     
            #Membaca semua daftar kombinasi sebelumnya
            for n in s:
                #Sorting untuk mencegah duplikasi
                r=tuple(sorted(set(j).difference(set(n))))
                l=len(n)
                #Check if len(n)==1 agar dapat mengekstrak "key" untuk mencari di support dictionaries  
                if(l==1):
                    n=value(n)
                    l=1
                if(len(r)==1):
                    r2=value(r)
                if(n!=None):
                    #Jika rules confidencenya lebih besar dari minconf, print the rule
                    if((ant[len(j)][j]/ant[l][n])>=minconf):
                        #Rules akan terlihat signifikan jika terdapat pada node tertutup,jika tidak akan berlebihan
                        if(closed.__contains__((n))):
                            c=c+1
                            if(len(r)==1):
                                a.append(n)
                                item_a.append(ant[l][n])
                                b.append(r2)
                                item_b.append(ant[len(r)][r2])
                                support.append((ant[len(j)][j]/len(data)))
                                confidence.append((ant[len(j)][j]/ant[l][n]))
                                lift_ratio.append(((ant[len(j)][j])/ant[l][n])/(ant[len(r)][r2])) 
                            else:
                                a.append(n)
                                item_a.append(ant[l][n])
                                b.append(r)
                                item_b.append(ant[len(r)][r])
                                support.append((ant[len(j)][j]/len(data)))
                                confidence.append((ant[len(j)][j]/ant[l][n]))
                                lift_ratio.append(((ant[len(j)][j])/ant[l][n])/(ant[len(r)][r]))

table_rules = pd.DataFrame({'Antecendent':a,
                            'Frequent_item_a':item_a,
                            'Consequent':b,
                            'Frequent_item_b':item_b,
                            'Support':support,
                            'Confidence':confidence,
                            'Lift_Ratio':lift_ratio
                           })

#Fungsi mengkali hasil tabel support dengan table confidence untuk keperluan menghitung kekuatan association rules
table_rules['SxC'] = table_rules['Support'] * table_rules['Confidence']

#Memanggil akhir hitungan execution time
t2 = time()
elapsed = t2 - t1

#Untuk memanggil hasil
table_rules

Masukkan nilai minimum support: 0.0045
Masukkan nilai minimum confidence: 0.2


,Antecendent,Frequent_item_a,Consequent,Frequent_item_b,Support,Confidence,Lift_Ratio,SxC
0,"(Cookies Hias, Nastar Spesial)",35,Dimsum Siomay Ayam,308,0.004866,0.228571,0.000742,0.001112
1,"(Cookies Hias, Dimsum Siomay Udang Keju)",47,Dimsum Siomay Ayam,308,0.006083,0.212766,0.000691,0.001294
2,"(Cookies Hias, Dimsum Siomay Ayam)",59,Dimsum Siomay Keju,328,0.009732,0.271186,0.000827,0.002639
3,"(Cookies Hias, Dimsum Siomay Keju)",51,Dimsum Siomay Ayam,308,0.009732,0.313725,0.001019,0.003053
4,"(Dimsum Siomay Ayam, Dimsum Siomay Keju)",63,Cookies Hias,286,0.009732,0.253968,0.000888,0.002472
...,...,...,...,...,...,...,...,...
103,Cookies Hias,286,Dimsum Siomay Ayam,308,0.035888,0.206294,0.000670,0.007403
104,Brownies Cookies,303,Dimsum Siomay Udang Keju,326,0.039538,0.214521,0.000658,0.008482
105,Sagu Keju,307,Dimsum Siomay Keju,328,0.038929,0.208469,0.000636,0.008116
106,Dimsum Siomay Ayam,308,Dimsum Siomay Keju,328,0.038321,0.204545,0.000624,0.007838


In [ ]:
print('Execution time : %f detik.' % elapsed)

In [ ]:
#Fungsi untuk menghitung kekuatan association rules
kekuatan = sum(table_rules['SxC']) / len(table_rules)

print ("Kekuatan Association Rules Algoritma FP-Growth : ", kekuatan )

In [ ]:
#Fungsi untuk menghitung jumlah nilai support dari hasil guna perhitungan akurasi algoritma
jumlah_support = sum(table_rules['Support'])

print ("Jumlah Nilai Support Algoritma FP-Growth : ", jumlah_support )